This exercise will require you to pull some data from the Qunadl API. Qaundl is currently the most widely used aggregator of financial market data.

As a first step, you will need to register a free account on the http://www.quandl.com website.

After you register, you will be provided with a unique API key, that you should store:

In [1]:
# Store the API key as a string - according to PEP8, constants are always named in all upper case
API_KEY = '' # put your api key here

Qaundl has a large number of data sources, but, unfortunately, most of them require a Premium subscription. Still, there are also a good number of free datasets.

For this mini project, we will focus on equities data from the Frankfurt Stock Exhange (FSE), which is available for free. We'll try and analyze the stock prices of a company called Carl Zeiss Meditec, which manufactures tools for eye examinations, as well as medical lasers for laser eye surgery: https://www.zeiss.com/meditec/int/home.html. The company is listed under the stock ticker AFX_X.

You can find the detailed Quandl API instructions here: https://docs.quandl.com/docs/time-series

While there is a dedicated Python package for connecting to the Quandl API, we would prefer that you use the *requests* package, which can be easily downloaded using *pip* or *conda*. You can find the documentation for the package here: http://docs.python-requests.org/en/master/ 

Finally, apart from the *requests* package, you are encouraged to not use any third party Python packages, such as *pandas*, and instead focus on what's available in the Python Standard Library (the *collections* module might come in handy: https://pymotw.com/3/collections/ ).
Also, since you won't have access to DataFrames, you are encouraged to us Python's native data structures - preferably dictionaries, though some questions can also be answered using lists.
You can read more on these data structures here: https://docs.python.org/3/tutorial/datastructures.html

Keep in mind that the JSON responses you will be getting from the API map almost one-to-one to Python's dictionaries. Unfortunately, they can be very nested, so make sure you read up on indexing dictionaries in the documentation provided above.

In [2]:
# First, import the relevant modules
import requests
import statistics #is part of Python Standard Library

In [3]:
# Now, call the Quandl API and pull out a small sample of the data (only one day) to get a glimpse
# into the JSON structure that will be returned
r = requests.get('https://www.quandl.com/api/v3/datasets/FSE/AFX_X/data.json?start_date=2019-10-01&end_date=2019-10-01&api_key={}'.format(API_KEY))
json_dict = r.json()
json_dict


{'dataset_data': {'limit': None,
  'transform': None,
  'column_index': None,
  'column_names': ['Date',
   'Open',
   'High',
   'Low',
   'Close',
   'Change',
   'Traded Volume',
   'Turnover',
   'Last Price of the Day',
   'Daily Traded Units',
   'Daily Turnover'],
  'start_date': '2019-10-01',
  'end_date': '2019-10-01',
  'frequency': 'daily',
  'data': [['2019-10-01',
    105.3,
    105.9,
    103.8,
    104.8,
    None,
    132282.0,
    13861037.0,
    None,
    None,
    None]],
  'collapse': None,
  'order': None}}

In [4]:
# Inspect the JSON structure of the object you created, and take note of how nested it is,
# as well as the overall structure
print('type(json_dict): ' + str(type(json_dict)))
print('type(json_dict[\'dataset_data\']): ' + str(type(json_dict['dataset_data'])))
print('type(json_dict[\'dataset_data\'][\'column_names\']): ' + str(type(json_dict['dataset_data']['column_names'])))
print('type(json_dict[\'dataset_data\'][\'data\']): ' + str(type(json_dict['dataset_data']['data'])))
print('type(json_dict[\'dataset_data\'][\'data\'][0]): ' + str(type(json_dict['dataset_data']['data'][0])))

type(json_dict): <class 'dict'>
type(json_dict['dataset_data']): <class 'dict'>
type(json_dict['dataset_data']['column_names']): <class 'list'>
type(json_dict['dataset_data']['data']): <class 'list'>
type(json_dict['dataset_data']['data'][0]): <class 'list'>


The object we get has dictionary type. All the information is contained under 'dataset_data' field. Inside, nested, we have dictionary describing fields of the response such as limit, order, start end end dates etc. Among other, we see nested array of column names, as well as "data" key, under which there is 2d array - array of info for each day we requested. Elements of the internal array correspond to column names stored under 'column_names' key.

These are your tasks for this mini project:

1. Collect data from the Franfurt Stock Exchange, for the ticker AFX_X, for the whole year 2017 (keep in mind that the date format is YYYY-MM-DD).
2. Convert the returned JSON object into a Python dictionary.
3. Calculate what the highest and lowest opening prices were for the stock in this period.
4. What was the largest change in any one day (based on High and Low price)?
5. What was the largest change between any two days (based on Closing Price)?
6. What was the average daily trading volume during this year?
7. (Optional) What was the median trading volume during this year. (Note: you may need to implement your own function for calculating the median.)

Let's collect data from the Franfurt Stock Exchange, for the ticker AFX_X, for the whole year 2017 and get .json field out of the response (it will be a python dictionary already):

In [5]:
# sending API request for the 2017 year
r = requests.get(
    "https://www.quandl.com/api/v3/datasets/FSE/AFX_X/data.json?start_date=2017-01-01&end_date=2017-12-31-&api_key={}".format(
        API_KEY
    )
)
# extracting json out of the responce. 
json_dict = r.json()

Now let's transform the data to the convenient structure: list of dictionaries, were keys of the dictionary corresponds to column names, and values - data from 'data' field. 

In [6]:
# getting list of cilumn names
columns = json_dict["dataset_data"]["column_names"]

# constructing list of dictionaries
data = [
    {columns: x for (columns, x) in zip(columns, x)}
    for x in json_dict["dataset_data"]["data"]
]

In [7]:
# examining our list (first 2 items)
data[:2]

[{'Date': '2017-12-29',
  'Open': 51.76,
  'High': 51.94,
  'Low': 51.45,
  'Close': 51.76,
  'Change': None,
  'Traded Volume': 34640.0,
  'Turnover': 1792304.0,
  'Last Price of the Day': None,
  'Daily Traded Units': None,
  'Daily Turnover': None},
 {'Date': '2017-12-28',
  'Open': 51.65,
  'High': 51.82,
  'Low': 51.43,
  'Close': 51.6,
  'Change': None,
  'Traded Volume': 40660.0,
  'Turnover': 2099024.0,
  'Last Price of the Day': None,
  'Daily Traded Units': None,
  'Daily Turnover': None}]

Now let's calculate what the highest and lowest opening prices were for the stock in this period.

In [8]:
# finding the element corresponding to the highest opening price
max(data, key=lambda x: x["Open"] if x["Open"] is not None else -float("inf"))

{'Date': '2017-12-14',
 'Open': 53.11,
 'High': 53.54,
 'Low': 52.15,
 'Close': 52.67,
 'Change': None,
 'Traded Volume': 132981.0,
 'Turnover': 7016953.0,
 'Last Price of the Day': None,
 'Daily Traded Units': None,
 'Daily Turnover': None}

In [9]:
# finding the element corresponding to the lowest opening price
min(data, key=lambda x: x["Open"] if x["Open"] is not None else float("inf"))

{'Date': '2017-01-24',
 'Open': 34.0,
 'High': 34.35,
 'Low': 33.85,
 'Close': 34.22,
 'Change': None,
 'Traded Volume': 48797.0,
 'Turnover': 1666086.0,
 'Last Price of the Day': None,
 'Daily Traded Units': None,
 'Daily Turnover': None}

Finding the largest change in any one day (based on High and Low price):

In [10]:
# the largest change in any one day
item = max(
    data,
    key=lambda x: x["High"] - x["Low"]
    if x["Open"] is not None and x["Close"] is not None
    else 0,
)
item

{'Date': '2017-05-11',
 'Open': 43.4,
 'High': 46.06,
 'Low': 43.25,
 'Close': 45.0,
 'Change': None,
 'Traded Volume': 189125.0,
 'Turnover': 8496322.0,
 'Last Price of the Day': None,
 'Daily Traded Units': None,
 'Daily Turnover': None}

In [11]:
print("The largest change in any one day:")
print(item["High"] - item["Low"])

The largest change in any one day:
2.8100000000000023


Finding the largest change between any two days (based on Closing Price):

In [12]:
# the largest change between any two days
a = [x["Close"] for x in data]
print("The largest change between any two days:")
max([abs(x - a[i - 2]) for i, x in enumerate(a) if i > 1])

The largest change between any two days:


3.1499999999999986

Finding the average daily trading volume during this year:

In [13]:
print('The average daily trading volume during this year:')
statistics.mean([x['Traded Volume'] for x in data])

The average daily trading volume during this year:


89124.33725490196

Finding the median trading volume during this year.

In [16]:
print('The median daily trading volume during this year:')
statistics.median([x['Traded Volume'] for x in data])

The median daily trading volume during this year:


76286.0